<a href="https://colab.research.google.com/github/biswajitburagohain/neural_net-practice/blob/master/create_real_world_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-05-02 15:50:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   244MB/s    in 0.6s    

2020-05-02 15:50:23 (244 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [0]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [0]:
train_horse_dir = os.path.join('/tmp/horse-or-human/horses') #dir with horse images
train_human_dir = os.path.join('/tmp/horse-or-human/humans') #dir with human images

In [4]:
train_horse_names = os.listdir(train_horse_dir) # list the names of the files in the dir
train_human_names = os.listdir(train_human_dir) # list the names of the files in the dir
print(train_horse_names[:3])
print('total horses in the subdir is: ' + str(len(train_horse_names)))

['horse07-7.png', 'horse10-4.png', 'horse49-6.png']
total horses in the subdir is: 500


In [0]:
########### training of the model starts here
import tensorflow as tf

In [0]:

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
                                   tf.keras.layers.Flatten(),
                                   tf.keras.layers.Dense(512, activation='relu'),
                                   tf.keras.layers.Dense(1, activation = 'sigmoid')])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 322624)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               1

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen = ImageDataGenerator(rescale = 1/255)

In [11]:
train_generator = train_datagen.flow_from_directory(directory='/tmp/horse-or-human',
                                                    target_size = (300,300),
                                                    class_mode = 'binary',
                                                    batch_size = 128)

Found 1027 images belonging to 2 classes.


In [12]:
model.fit(train_generator, steps_per_epoch=8, epochs=15, verbose=1)

Epoch 1/15
8/8 [==============================] - 6s 724ms/step - loss: 51.2617 - accuracy: 0.5107
Epoch 2/15
8/8 [==============================] - 6s 787ms/step - loss: 2.1407 - accuracy: 0.6808
Epoch 3/15
8/8 [==============================] - 6s 700ms/step - loss: 0.3472 - accuracy: 0.8687
Epoch 4/15
8/8 [==============================] - 6s 691ms/step - loss: 1.2255 - accuracy: 0.7608
Epoch 5/15
8/8 [==============================] - 6s 697ms/step - loss: 0.2921 - accuracy: 0.8710
Epoch 6/15
8/8 [==============================] - 6s 691ms/step - loss: 0.4038 - accuracy: 0.8932
Epoch 7/15
8/8 [==============================] - 6s 699ms/step - loss: 0.1704 - accuracy: 0.9143
Epoch 8/15
8/8 [==============================] - 6s 705ms/step - loss: 0.3446 - accuracy: 0.9032
Epoch 9/15
8/8 [==============================] - 5s 686ms/step - loss: 0.0305 - accuracy: 0.9900
Epoch 10/15
8/8 [==============================] - 6s 765ms/step - loss: 0.0061 - accuracy: 0.9990
Epoch 11/15
8/8 [=

In [16]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving horse.jpg to horse (2).jpg
[0.]
horse.jpg is a horse
